In [ ]:
!pip install utils
!pip install unet
!pip install h5py

In [ ]:
!pip install --upgrade tensorflow
#print(tf.__version__)

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
import h5py
import tqdm

import cv2
import skimage.io
import skimage.transform
import skimage.morphology
import skimage.color
import sklearn.model_selection
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras import metrics
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from utils import *
# from unet import *

In [ ]:
#Size parameters of image
im_width = 256
im_height = 256

In [ ]:
#Create two lists for mask image and tif image
#1. list of all files containing word 'mask' 
image_filenames_train = []
mask_files = glob('/kaggle/input/lgg-mri-segmentation/kaggle_3m/*/*_mask*')

#list of train files not containing 'mask'
for i in mask_files:
  image_filenames_train.append(i.replace('_mask', ''))

print(image_filenames_train[:10])
len(image_filenames_train)

In [ ]:
def plot_from_img_path(rows,columns, list_img_path, list_mask_path):
    fig = plt.figure(figsize = (12,12))
    for i in range(1,rows * columns + 1):
        fig.add_subplot(rows, columns, i)
        img_path = list_img_path[i]
        mask_path = list_mask_path[i]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path)
        plt.imshow(image)
        plt.imshow(mask, alpha =0.4)
    plt.show()
        

In [ ]:
plot_from_img_path(3, 3, image_filenames_train, mask_files)

**Creating Data frame & Split data on Train set, Validation set , Test set**

In [ ]:
df = pd.DataFrame(data={'image_filenames_train':image_filenames_train,'mask':mask_files})
#split training and test data. 10% for test data
df_train, df_test = train_test_split(df, test_size=0.1)

#split training and validation data. 20% for validation data
df_train, df_val = train_test_split(df_train, test_size=0.2)

print(df_train.shape)
print(df_test.shape)
print(df_val.shape)


**Data Generator, Data Augmentation and adjust data**
Data Generator - prepare dataset to pass data in batches to the model.
Data Augmentation  - applying transformations to data.(Increase size & quality of dataset)

In [ ]:
#Referencing code from- https://github.com/zhixuhao/unet/blob/master/data.py
def train_generator(
    data_frame,
    batch_size,
    augmentation_dict,
    image_color_mode="rgb",
    mask_color_mode="grayscale",
    image_save_prefix="image",
    mask_save_prefix="mask",
    save_to_dir=None,
    target_size=(256, 256),
    seed=1,
):
    """
    can generate image and mask at the same time use the same seed for
    image_datagen and mask_datagen to ensure the transformation for image
    and mask is the same if you want to visualize the results of generator,
    set save_to_dir = "your path"
    """
    image_datagen = ImageDataGenerator(**augmentation_dict)
    mask_datagen = ImageDataGenerator(**augmentation_dict)

    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col="image_filenames_train",
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix,
        seed=seed,
    )

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col="mask",
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix,
        seed=seed,
    )

    train_gen = zip(image_generator, mask_generator)
    
    # Final return Tuple after image Normalization and Diagnostics
    for (img, mask) in train_gen:
        img, mask = normalize_and_diagnose(img, mask)
        yield (img, mask)

In [ ]:
#After normalization or adjusting data , if value is <=0.5 then mask 
#is complete black and it does not have tumor
def normalize_and_diagnose(img, mask):
    img = img/255
    mask = mask/255
    mask[mask>0.5] = 1
    mask[mask<=0.5] = 0
    return (img,mask)

**Utility Function for dice loss**

In [ ]:
def dice_coefficients(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred) 
    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2*intersection+smooth)/ (union+smooth)

def dice_coefficient_loss(y_true, y_pred, smooth=100):
    return -dice_coefficients(y_true, y_pred, smooth)

#iou - intersection over union
def iou(y_true, y_pred, smooth=100):
    intersection = K.sum(y_true * y_pred)
    sum = K.sum(y_true + y_pred)
    iou = (intersection+smooth)/(sum-intersection+smooth)
    return iou

def jaccard_distance(y_true, y_pred):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    return -iou(y_true_flatten, y_pred_flatten)

**UNet Model Architecture**

In [ ]:
def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    
    #Encoding leg
    conv1 = Conv2D(filters=64, kernel_size =(3,3), padding="same")(inputs)
    batchNorm1 = Activation("relu")(conv1)
    conv1 = Conv2D(filters=64, kernel_size =(3,3), padding="same")(batchNorm1)
    batchNorm1 = BatchNormalization(axis=3)(conv1)
    batchNorm1 = Activation("relu")(batchNorm1)
    pool1 = MaxPooling2D(pool_size=(2,2))(batchNorm1)
    
    conv2 = Conv2D(filters=128, kernel_size =(3,3), padding="same")(pool1)
    batchNorm2 = Activation("relu")(conv2)
    conv2 = Conv2D(filters=128, kernel_size =(3,3), padding="same")(batchNorm2)
    batchNorm2 = BatchNormalization(axis=3)(conv2)
    batchNorm2 = Activation("relu")(batchNorm2)
    pool2 = MaxPooling2D(pool_size=(2,2))(batchNorm2)
    
    conv3 = Conv2D(filters=256, kernel_size =(3,3), padding="same")(pool2)
    batchNorm3 = Activation("relu")(conv3)
    conv3 = Conv2D(filters=256, kernel_size =(3,3), padding="same")(batchNorm3)
    batchNorm3 = BatchNormalization(axis=3)(conv3)
    batchNorm3 = Activation("relu")(batchNorm3)
    pool3 = MaxPooling2D(pool_size=(2,2))(batchNorm3)
    
    conv4 = Conv2D(filters=512, kernel_size =(3,3), padding="same")(pool3)
    batchNorm4 = Activation("relu")(conv4)
    conv4 = Conv2D(filters=512, kernel_size =(3,3), padding="same")(batchNorm4)
    batchNorm4 = BatchNormalization(axis=3)(conv4)
    batchNorm4 = Activation("relu")(batchNorm4)
    pool4 = MaxPooling2D(pool_size=(2,2))(batchNorm4)
    
    conv5 = Conv2D(filters=1024, kernel_size =(3,3), padding="same")(pool4)
    batchNorm5 = Activation("relu")(conv5)
    conv5 = Conv2D(filters=1024, kernel_size =(3,3), padding="same")(batchNorm5)
    batchNorm5 = BatchNormalization(axis=3)(conv5)
    batchNorm5 = Activation("relu")(batchNorm5)
    
    #Upconvolution/Decoder Leg - starts Conv2DTranspose
    #u6=u6+c4
    up6 = concatenate([Conv2DTranspose(512,kernel_size=(2,2),strides=(2,2),padding="same")(batchNorm5),conv4,],axis=3,)
    conv6 = Conv2D(filters=512, kernel_size=(3,3),padding="same")(up6)
    batchNorm6 = Activation("relu")(conv6)
    conv6 = Conv2D(filters=512, kernel_size=(3,3),padding="same")(batchNorm6)
    batchNorm6 = BatchNormalization(axis=3)(conv6)
    batchNorm6 = Activation("relu")(batchNorm6)
    
     #u7=u7+c3
    up7 = concatenate([Conv2DTranspose(256,kernel_size=(2,2),strides=(2,2),padding="same")(batchNorm6),conv3,],axis=3,)
    conv7 = Conv2D(filters=256, kernel_size=(3,3),padding="same")(up7)
    batchNorm7 = Activation("relu")(conv7)
    conv7 = Conv2D(filters=256, kernel_size=(3,3),padding="same")(batchNorm7)
    batchNorm7 = BatchNormalization(axis=3)(conv7)
    batchNorm7 = Activation("relu")(batchNorm7)
    
    #u8=u8+c2
    up8 = concatenate([Conv2DTranspose(128,kernel_size=(2,2),strides=(2,2),padding="same")(batchNorm7),conv2,],axis=3,)
    conv8 = Conv2D(filters=128, kernel_size=(3,3),padding="same")(up8)
    batchNorm8 = Activation("relu")(conv8)
    conv8 = Conv2D(filters=128, kernel_size=(3,3),padding="same")(batchNorm8)
    batchNorm8 = BatchNormalization(axis=3)(conv8)
    batchNorm8 = Activation("relu")(batchNorm8)
    
    #u9=u9+c1
    up9 = concatenate([Conv2DTranspose(64,kernel_size=(2,2),strides=(2,2),padding="same")(batchNorm8),conv1,],axis=3,)
    conv9 = Conv2D(filters=64, kernel_size=(3,3),padding="same")(up9)
    batchNorm9 = Activation("relu")(conv9)
    conv9 = Conv2D(filters=64, kernel_size=(3,3),padding="same")(batchNorm9)
    batchNorm9 = BatchNormalization(axis=3)(conv9)
    batchNorm9 = Activation("relu")(batchNorm9)
    
    conv10 = Conv2D(filters=1, kernel_size=(1,1),activation ="sigmoid")(batchNorm9)
    
#     if(pretrained_weights):
#     	model.load_weights(pretrained_weights)
        
    return Model(inputs=[inputs],outputs=[conv10])

In [ ]:
#Define hyperparameters
EPOCHS = 25
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
smooth =100

**Main- Train the model**

In [ ]:
model = unet(input_size=(im_height, im_width, 3))

model.summary()

In [ ]:
train_generator_param = dict(rotation_range= 0.2, width_shift_range=0.05,height_shift_range=0.05,shear_range=0.05, zoom_range=0.05, horizontal_flip=True,fill_mode ='nearest' )
train_gen = train_generator(df_train,BATCH_SIZE,train_generator_param,target_size=(im_height,im_width))
test_gen = train_generator(df_val,BATCH_SIZE, dict(),target_size=(im_height,im_width))

train_steps = len(df_train)//BATCH_SIZE
valid_steps = len(df_val)//BATCH_SIZE

decay_rate = LEARNING_RATE/EPOCHS
adam = tf.keras.optimizers.Adam(lr=LEARNING_RATE,beta_1=0.9,beta_2=0.999,epsilon=None,amsgrad=False,weight_decay=decay_rate)
# optimizer.build(Adam(lr=LEARNING_RATE,beta_1=0.9,beta_2=0.999,epsilon=None,amsgrad=False,weight_decay=decay_rate))
callbacks = [ModelCheckpoint('unetArch.hdf5', verbose=2,save_best_only=True)]

model.compile(optimizer="adam", loss=dice_coefficient_loss, metrics=["binary_accuracy", iou, dice_coefficients])

history =model.fit(train_gen,batch_size=BATCH_SIZE,epochs=EPOCHS,
          callbacks = callbacks, validation_data=test_gen, steps_per_epoch=train_steps, 
          validation_steps=valid_steps)

In [ ]:
import pprint
pprint.pprint(history.history)

In [ ]:
history_post_training = history.history

train_dice_coeff = history_post_training['dice_coefficient']
#dice_coefficient & val_dice_coefficient are accessed from history
test_dice_coeff = history_post_training['val_dice_coefficient']

#Jaccard distance for plotting accuracy
train_jaccard_list = history_post_training['iou']
test_jaccard_list = history_post_training['val_iou']

# Loss for plotting loss
train_loss_list = history_post_training['loss']
test_loss_list = history_post_training['val_loss']

plt.figure(1)
# Epochs VS Loss
plt.plot(test_loss_list, 'b-')
plt.plot(train_loss_list, 'r-')

plt.xlabel('iterations')   #epochs
plt.ylabel('loss')
plt.title('Loss Graph', fontsize =12)

plt.figure(2)
# Epochs VS Accuracy
plt.plot(train_dice_coeff, 'b-')
plt.plot(test_dice_coeff, 'r-')

plt.xlabel('iterations')   #epochs
plt.ylabel('accuracy')
plt.title('Accuracy Graph', fontsize =12)
plt.show()

**Apply trained model in test data set**

In [ ]:
#Load previously trained model from 'unetArch.hdf5' file
model = load_model(unetArch.hdf5, custome_objects = {'dice_coefficient_loss':dice_coefficients_loss,'iou':iou,'dice_coefficient':dice_coefficients})

In [ ]:
test_gen = train_generator(df_test,BATCH_SIZE, dict(),target_size=(im_height,im_width))

results = model.evaluate(test_gen, steps= len(df_test)//BATCH_SIZE)
print("Test loss: ", results[0])
print("Test IoU: ", results[1])
print("Test Dice Coefficient: ", results[2])

**"Final Prediction on Test data"
Plotting Predicted Masks Segmentation Results from Test image set**

In [ ]:
for i in range (20):
    index = np.random.ranint(1,len(df_test.index))
    img = cv2.imread(df_test['image_filenames_train'].iloc[index])
    img = cv2.resize(img, (im_height,im_width))
    img = img/255   #normalize
    #print(img.shape)     (256,256,3) ->3D tensor
    #To get 4D array
    img = img[np.newaxis, :, :, :]
    #print(img.shape)     (1,256,256,3) ->4D array
    
    predicted_img = model.predict(img)
    plt.figure(figsize=(12,12))
    
    #Plot original image, original mask iamge , predicted image
    plt.subplot([1,3,1])   #1row 3coloumn 1st index
    plt.imshow(np.squeeze(img))
    plt.title('Original image')
    
    plt.subplot([1,3,2])   #1row 3coloumn 2nd index
    plt.imshow(np.squeeze(cv2.imread(df_test['mask'].iloc[index])))
    plt.title('Original mask')
    
    plt.subplot([1,3,3])   #1row 3coloumn 3rd index
    plt.imshow(np.squeeze(predicted_img)>0.5)
    plt.title('Predicted image')
    